In [ ]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
G = 0.1
dt = 0.01

In [ ]:
class CosmicBody:
    def __init__(self, mass: float, r: np.ndarray, speed: np.ndarray, name: str):
        self.mass = mass
        self.speed = speed
        self.r = r
        self.trajectory = None
        self.history_r = np.array(self.r, dtype=np.float64)
        self.name = name
